# Labeling of Incidents

## Configuration file

In [1]:
configuration = 'predicting.yaml'

In [2]:
import os, sys
src_dir = os.environ['BASKERVILLE_ROOT'] + '/src'
if not os.path.lexists(src_dir):
    raise RuntimeError('Baskerville source dir does not exist!')
module_path = os.path.abspath(os.path.join(src_dir))
if module_path not in sys.path:
    sys.path.append(module_path)

## Attack parameters

In [3]:
import time
import pandas as pd
import numpy as np
from sklearn.ensemble import GradientBoostingClassifier
from sklearn.preprocessing import StandardScaler
import matplotlib.pyplot as plt
%matplotlib inline
from baskerville.util.helpers import parse_config
from baskerville.util.baskerville_tools import BaskervilleDBTools
from baskerville.models.config import BaskervilleConfig
from baskerville.db import set_up_db
from baskerville.db.models import Attack, Attribute, RequestSet, Model  
from baskerville.models.engine import BaskervilleAnalyticsEngine

/usr/local/lib/python3.6/site-packages/sqlalchemy/sql/functions.py:67: SAWarning: The GenericFunction 'array_agg' is already registered and is going to be overridden.
  "is going to be overridden.".format(identifier)
/usr/local/lib/python3.6/site-packages/psycopg2/__init__.py:144: UserWarning: The psycopg2 wheel package will be renamed from release 2.8; in order to keep installing from binary please use "pip install psycopg2-binary" instead. For details see: <http://initd.org/psycopg/docs/install.html#binary-install-from-pypi>.
  """)


In [4]:
os.environ['TZ'] = 'UTC'
time.tzset()
conf_path = os.environ['BASKERVILLE_ROOT'] + f'/conf/{configuration}'
config = parse_config(path=conf_path)
config = BaskervilleConfig(config).validate()
features = config.engine.extra_features

config.database.maintenance = None
db_tools = BaskervilleDBTools(config.database)
db_tools.connect_to_db()

2021-01-19 00:34:12,024 baskerville.models.config.validate +208: DEBUG    [25912] Validating BaskervilleConfig...
2021-01-19 00:34:12,031 baskerville.models.config.validate +688: DEBUG    [25912] Validating DatabaseConfig...
2021-01-19 00:34:12,036 baskerville.models.config.validate +307: DEBUG    [25912] Validating EngineConfig...
2021-01-19 00:34:12,040 baskerville.models.config.validate +455: DEBUG    [25912] Validating SimulationConfig...
2021-01-19 00:34:12,048 baskerville.models.config.validate +966: DEBUG    [25912] Validating DataParsingConfig...
2021-01-19 00:34:12,438 baskerville.models.config.validate +938: DEBUG    [25912] Validating MetricsConfig...
2021-01-19 00:34:12,442 baskerville.models.config.validate +490: DEBUG    [25912] Validating TrainingConfig...
/root/baskerville/src/baskerville/models/config.py:340: UserWarning: Time bucket is NOT set to the default 120 seconds.
  warnings.warn('Time bucket is NOT set to the default 120 seconds.')
2021-01-19 00:34:12,450 bask

## Attack parameters

In [1]:
# target = '1'
# table = 'request_sets_y2020_w14'
# fileter_column = 'stop'
# attack_start = '2020-04-03 09:49:00'
# attack_stop = '2020-04-03 10:49:00'
# regular_start = '2020-03-22 00:00:00'
# regular_stop = '2020-04-03 09:00:00'

# target = '2'
# table = 'request_sets_y2020_w20'
# fileter_column = 'stop'
# attack_start = '2020-05-11 21:06:00'
# attack_stop = '2020-05-11 21:18:00'
# regular_start = '2020-05-11 00:00:00'
# regular_stop = '2020-05-11 21:00:00'

# target = '3'
# table = 'request_sets_y2020_w21'
# fileter_column = 'stop'
# attack_start = '2020-05-20 22:20:00'
# attack_stop = '2020-05-20 22:40:00'
# regular_start = '2020-05-20 18:00:00'
# regular_stop = '2020-05-20 22:00:00'

# target = '4'
# table = 'request_sets_y2020_w24'
# fileter_column = 'stop'
# attack_start = '2020-06-08 14:00:00'
# attack_stop = '2020-06-08 14:08:00'
# regular_start = '2020-06-05 18:00:00'
# regular_stop = '2020-06-08 13:00:00'

# target = '5'
# table = 'request_sets_y2020_w24'
# fileter_column = 'stop'
# attack_start = '2020-06-12 11:56:00'
# attack_stop = '2020-06-12 12:02:00'
# regular_start = '2020-06-12 00:00:00'
# regular_stop = '2020-06-12 11:52:00'

# target = '6'
# fileter_column = 'stop'
# table = 'request_sets_y2020_w27'
# attack_start = '2020-07-05 14:28:00Z'
# attack_stop = '2020-07-05 14:46:00Z'
# regular_start = '2020-07-04 14:46:00Z'
# regular_stop = '2020-07-05 09:46:00Z'

# target = '7'
# table = 'request_sets'
# fileter_column = 'created_at'
# attack_start = '2020-07-09 08:34:00Z'
# attack_stop = '2020-07-09 09:56:00Z'
# regular_start = '2020-07-09 00:00:00Z'
# regular_stop = '2020-07-09 08:00:00Z'

# target = '8'
# table = 'request_sets'
# fileter_column = 'stop'
# attack_start = '2020-07-31 00:00:00Z'
# attack_stop = '2020-07-31 00:30:00Z'
# regular_start = '2020-07-31 02:02:00Z'
# regular_stop = '2020-07-31 10:00:00Z'

# id 18 - classifier
# id 19 - vladimir's list of ips
# target = '9'
# table = 'request_sets'
# fileter_column = 'stop'
# attack_start = '2020-09-17 17:47:00Z'
# attack_stop = '2020-09-17 17:59:00Z'
# regular_start = '2020-09-17 16:00:00Z'
# regular_stop = '2020-09-17 17:00:00Z'

# target = '10'
# table = 'request_sets'
# fileter_column = 'stop'
# attack_start = '2020-10-05 19:46:00Z'
# attack_stop = '2020-10-05 20:18:00Z'
# regular_start = '2020-10-05 15:00:00Z'
# regular_stop = '2020-10-05 19:00:00Z'

# target = '11'
# table = 'request_sets'
# fileter_column = 'stop'
# attack_start = '2020-10-14 10:20:00Z'
# attack_stop = '2020-10-14 10:24:00Z'
# regular_start = '2020-10-14 00:00:00Z'
# regular_stop = '2020-10-14 09:00:00Z'

# target = 'bunnyapps'  
# table = 'request_sets'
# fileter_column = 'stop'
# attack_start = '2020-10-22 08:48:00Z'
# attack_stop = '2020-10-22 09:02:00Z'
# regular_start = '2020-10-01 00:00:00Z'
# regular_stop = '2020-10-22 05:00:00Z'

# target = '12'
# table = 'request_sets'
# fileter_column = 'stop'
# attack_start = '2020-10-22 09:37:00Z'
# attack_stop = '2020-10-22 09:41:00Z'
# regular_start = '2020-10-01 00:00:00Z'
# regular_stop = '2020-10-22 05:00:00Z'

table = 'request_sets'
fileter_column = 'stop'

# target = '13'   24
# attack_start = '2021-01-14 14:36:00Z'
# attack_stop = '2021-01-14 15:36:00Z'
# regular_start = '2021-01-14 10:00:00Z'
# regular_stop = '2021-01-14 14:35:00Z'



# target = '14'  not saved, not a clear attack
# attack_start = '2021-01-14 16:30:00Z'
# attack_stop = '2021-01-14 18:00:00Z'
# regular_start = '2021-01-14 00:00:00Z'
# regular_stop = '2021-01-14 16:25:00Z'

# target = '15'  # id 25 'low and slow' manual list from Vladimir
# attack_start = '2021-01-17 16:44:00Z'
# attack_stop = '2021-01-17 18:00:00Z'
# regular_start = '2021-01-17 00:00:00Z'
# regular_stop = '2021-01-17 16:30:00Z'

target = '16'  # id 26
attack_start = '2021-01-26 08:35:00Z'
attack_stop = '2021-01-26 08:47:00Z'
regular_start = '2021-01-26 00:00:00Z'
regular_stop = '2021-01-26 08:30:00Z'

## Load Dataset

In [7]:
def load_dataset(query, db_tools):
    print('Querying database...')
    df = pd.read_sql(query, db_tools.engine)
    print(f'{len(df)} records retrieved.')
    num_ips = len(df['ip'].unique())
    print(f'{num_ips} unique IPs')
    print(f'Unwrapping features...')
    if len(df) > 0:
        df[[*df['features'][0].keys()]] = df[
            'features'
        ].apply(pd.Series)
        df.drop('features', axis=1, inplace=True)
        df.drop('host', axis=1, inplace=True)
        df.drop('country', axis=1, inplace=True)
        df.drop('request_total', axis=1, inplace=True)
        df.drop('minutes_total', axis=1, inplace=True)
    print(f'Unwrapping features complete.')
    df = df.fillna(0)
    return df

In [8]:
df_attack = load_dataset(f'select * from {table} where target=\'{target}\' and ' \
                                 f'{fileter_column} > \'{attack_start}\' and '\
                                 f'{fileter_column} < \'{attack_stop}\'', db_tools)

Querying database...
12136 records retrieved.
3343 unique IPs
Unwrapping features...
Unwrapping features complete.


In [9]:
df_regular = load_dataset(f'select * from {table} where target=\'{target}\' and ' \
                                 f'{fileter_column} > \'{regular_start}\' and '\
                                 f'{fileter_column} < \'{regular_stop}\'', db_tools)

Querying database...
22137 records retrieved.
5790 unique IPs
Unwrapping features...
Unwrapping features complete.


In [10]:
regular_ips = df_regular[['ip']].drop_duplicates()
df_attack=df_attack.merge(regular_ips[['ip']], on=['ip'], how='outer', indicator=True)
df_attack=df_attack[df_attack['_merge']=='left_only']

In [11]:
len(df_attack['ip'].unique())

3127

In [12]:
len(df_regular['ip'].unique())

5790

## Classifier

In [13]:
if 'host' in features:
    features.remove('host')
if 'country' in features:
    features.remove('country')
if 'host_country' in features:
    features.remove('host_country')
if 'request_total' in features:
    features.remove('request_total')
if 'minutes_total' in features:
    features.remove('minutes_total')

labels = np.ones(len(df_regular), dtype=int)
labels = np.append(labels, np.zeros(len(df_attack), dtype=int))
dataset = pd.concat([df_regular[features], df_attack[features]])

# scaler = StandardScaler()
# dataset = scaler.fit_transform(dataset[features].values)

model = GradientBoostingClassifier(
    n_estimators=500, random_state=777, 
    max_depth=12,
    max_features='auto',
    learning_rate = 0.05)
model.fit(dataset, labels)

#df_attack_scaled = scaler.transform(df_attack[features])
# predictions = model.predict(df_attack_scaled)

predictions = model.predict(df_attack[features])

In [14]:
incident = df_attack[['ip']].copy()
incident['predictions'] = predictions
attackers_predicted = incident[incident['predictions'] == 0][['ip']].drop_duplicates()
regulars_predicted = incident[incident['predictions'] == 1][['ip']].drop_duplicates()

incident_ips = incident[['ip']].drop_duplicates()
regular_ips = df_regular[['ip']].drop_duplicates()

attackers_vs_regular_traffic = pd.merge(regular_ips, attackers_predicted, how='inner', on=['ip'])
regulars_vs_incident = pd.merge(regular_ips, incident_ips, how='inner', on=['ip'])

print(f'Number of unique IPs in the incident = {len(incident_ips)}')
print(f'Number of predicted attackers = {len(attackers_predicted)}')
print(f'Number of predicted regulars = {len(incident_ips)-len(attackers_predicted)}')
print(f'Intersection predicted attackers in regular traffic = {len(attackers_vs_regular_traffic)}, {len(attackers_vs_regular_traffic)/len(regular_ips)*100:2.1f}%')
print(f'Intersection regular traffic in incident = {len(regulars_vs_incident)}')

# filter out the IPs from regular traffic
attackers = pd.merge(attackers_predicted, regulars_vs_incident, how='outer', on=['ip'], indicator=True)
attackers = attackers[attackers['_merge'] == 'left_only'][['ip']]

print(f'Final number of attackers IP: {len(attackers)}')

Number of unique IPs in the incident = 3127
Number of predicted attackers = 3118
Number of predicted regulars = 9
Intersection predicted attackers in regular traffic = 0, 0.0%
Intersection regular traffic in incident = 0
Final number of attackers IP: 3118


## Save Attack to DB

In [15]:
# attackers = df_attack[['ip']].drop_duplicates()

In [16]:
attack = Attack()
attack.start = attack_start
attack.stop = attack_stop
attack.target = target

attributes = []
for ip in attackers['ip']:
    a = Attribute()
    a.value = ip
    a.attacks.append(attack)
    attributes.append(a)

db_tools.session.add(attack)
db_tools.session.add_all(attributes)
db_tools.session.commit()
print(f'New attack saved in the database. id = {attack.id}')

New attack saved in the database. id = 26


## Save Attack (hardcoded list of IPs)

In [2]:
manual_ips = []

In [26]:
attack = Attack()
attack.start = attack_start
attack.stop = attack_stop
attack.target = target
attributes = []
for ip in manual_ips:
    a = Attribute()
    a.value = ip
    a.attacks.append(attack)
    attributes.append(a)
db_tools.session.add(attack)
db_tools.session.add_all(attributes)
db_tools.session.commit()

In [27]:
attack.id

25

In [3]:
banjax_ips = []

In [53]:
len(banjax_ips)

301

In [54]:
attack_ips = attackers['ip'].values.tolist()

In [55]:
len(attack_ips)

7584

In [56]:
len(banjax_ips and attack_ips)

7584

In [59]:
len(list(set(banjax_ips) & set(attack_ips)))

296